<a href="https://colab.research.google.com/github/Hemanthnaidu3737/Sentiment-Analysis-using-BERT-and-Transformers/blob/main/SENTI_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import gradio as gr
from transformers import pipeline
import torch
import time
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Enable GPU if available
device = 0 if torch.cuda.is_available() else -1

In [4]:
# Keyword lists
positive_keywords = [
    "happy", "joy", "love", "awesome", "great", "fantastic", "thank you",
    "excited", "peace", "satisfied", "grateful", "amazing", "cool", "positive",
    "brilliant", "nice", "sweet", "adorable", "hopeful", "cheerful", "blessed",
    "like", "hope", "smile", "wonderful", "excellent", "lovely", "beautiful",
    "friendly", "enjoy", "delightful", "fabulous", "nice work", "well done",
    "encouraging", "proud", "motivated", "inspired", "optimistic", "uplifting",
    "joyful", "awesome job", "best", "glad", "thankful", "victory", "success",
    "charming", "supportive", "respectful", "calm", "golden", "celebrate", "champion",
    "content", "peaceful", "harmony", "radiant", "energetic", "generous", "lucky",
    "win", "progress", "growth", "fun", "genuine", "warm", "affection", "devoted",
    "smiling", "kind", "loving", "appreciated", "healing", "strong", "determined",
    "confident", "playful", "admirable", "loyal", "mindful", "caring", "heartwarming",
    "positive vibes", "good job", "you're amazing", "way to go", "phenomenal",
    "outstanding", "you rock", "sweetheart", "heartfelt", "pleased", "bright", "upbeat",
    "graceful", "dream big", "be proud", "joy inside"
]



negative_keywords = [
    "sad", "lonely", "hopeless", "depressed", "anxious", "afraid", "angry", "hate",
    "furious", "suicidal", "kill myself", "die", "worthless", "failure", "panic",
    "murder", "crime", "violence", "hurt", "awful", "terrible", "pain", "suffering",
    "burnout", "exhausted", "guilty", "ashamed", "cut", "self-harm", "frustrated",
    "suffer", "f*ck", "sh*t", "broken", "crying", "giving up", "hate myself", "numb",
    "despair", "stressed", "rejected", "abandoned", "unloved", "useless", "can’t take it",
    "end it all", "overwhelmed", "mental breakdown", "trapped", "down", "devastated", "rude",
    "hopelessness", "darkness", "rage", "fear", "tears", "dying", "lost", "empty",
    "helpless", "no one cares", "screaming", "unworthy", "heartbroken", "breakdown",
    "isolation", "destructive", "lifeless", "worthless piece", "hate everyone",
    "no reason to live", "what's the point", "not enough", "nobody", "alone", "abuse",
    "neglected", "trauma", "shattered", "toxic", "downfall", "worse", "defeated",
    "burden", "sorrow", "cry", "low", "messed up", "can't breathe", "hurting",
    "emotional pain", "beyond help", "dark thoughts", "unfixable", "hell", "ugly",
    "insecure", "shame", "regret", "can't cope", "falling apart", "unseen", "ignored",
    "blame", "failure again", "rejected again", "numbness", "nightmare"
]



neutral_keywords = [
    "okay", "alright", "fine", "neutral", "average", "normal", "typical", "standard",
    "maybe", "not sure", "possibly", "perhaps", "hmm", "k", "noted", "whatever",
    "doesn’t matter", "no opinion", "sometimes", "depends", "neither", "in between",
    "it is what it is", "okayish", "meh", "so-so", "I guess", "more or less", "generally",
    "can you", "how to", "what if", "what about", "not bad", "nothing special", "nothing much",
    "informative", "understood", "not important", "it's fine", "acceptable", "moderate",
    "decent", "typical day", "regular", "same as usual", "just wondering", "as expected",
    "let's see", "not a big deal", "satisfactory", "unclear", "ambiguous", "somehow",
    "uncertain", "maybe later", "eventually", "I don’t know", "could be", "probably",
    "in general", "for now", "let me think", "guess so", "alright then", "heard that",
    "reasonable", "I suppose", "tolerable", "low-key", "casual", "no change", "unchanged",
    "not really", "little change", "non-emotional", "regular update", "status quo",
    "can’t say", "more or less fine", "typical response", "just saying", "on hold",
    "need details", "just checking", "waiting", "need more info", "technical", "background info",
    "not now", "perhaps later", "no rush", "sometime", "pause", "in progress", "partial",
    "random", "neutral tone", "some updates", "non-specific", "okay response", "so be it"
]

suicidal_keywords = [
    "suicidal", "kill myself", "end it all", "no reason to live", "die", "dying",
    "can't take it", "give up on life", "hate myself", "not worth it", "I want to die",
    "jump off", "overdose", "goodbye forever", "self-harm", "cut myself", "done living",
    "tired of life", "life is hell", "end my pain", "life is meaningless", "nothing to live for",
    "taking my life", "want to disappear", "I quit", "ending it", "goodbye world",
    "I wish I wasn’t born", "won’t wake up", "ending myself", "final goodbye", "last day",
    "life hurts", "I want out", "ending soon", "death is better", "ready to die"
]





# Master function to check intent
def detect_emotion_intent(text):
    text_lower = text.lower()

    if any(word in text_lower for word in suicidal_keywords):
        return "suicidal"
    elif any(word in text_lower for word in negative_keywords):
        return "negative"
    elif any(word in text_lower for word in positive_keywords):
        return "positive"
    elif any(word in text_lower for word in neutral_keywords):
        return "neutral"
    else:
        return "unknown"  # In case no keywords matched



In [5]:
# Load pre-trained sentiment analysis model
sentiment_pipeline = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment", device=device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [6]:
# Define intent-based responses
intent_responses = {
    "positive": [
        "That's great to hear! 😊",
        "Awesome! Keep spreading positivity! 🎉",
        "You’re shining bright today! 🌟",
        "Love the good vibes you're giving off! 💖",
        "That's a wonderful feeling to have. Keep it up! 🌈",
        "You’re doing amazing — be proud of yourself! 🙌",
        "Glad to hear things are going well for you. ✨",
        "Your energy is contagious! 🔥",
        "That's a beautiful emotion to share. 💫",
        "Keep riding that wave of joy! 🏄‍♂️"
    ],

    "neutral": [
        "Okay, got it. Thanks for sharing! 😐",
        "Alright, I understand. Let’s keep going. ➡️",
        "Thanks for the update. 📋",
        "Noted. If anything changes, feel free to share. 🧠",
        "That sounds fair. Let’s see how it unfolds. 🔄",
        "Alright then, moving forward! 🚶",
        "Seems like a balanced view. 🎯",
        "Appreciate the input! 👍",
        "Taking note of that. 📝",
        "Gotcha. Let me know if there's more. 🤖"
    ],

    "negative": [
        "I'm really sorry you're feeling this way. 😔",
        "That sounds tough. You're not alone. 🫂",
        "Take a deep breath — you're stronger than you think. 💪",
        "It’s okay to feel down sometimes. I’m here with you. 🌧️",
        "Sending some positive thoughts your way. 🌻",
        "I’m here for you. Let’s take this one step at a time. 👣",
        "I believe in your strength to overcome this. 🦋",
        "You’ve been through a lot — be kind to yourself. 🤍",
        "Remember, after rain comes the rainbow. 🌦️",
        "You’re not alone — feel free to share more. 🧡"
    ],

    "suicidal": [
        "I'm really sorry you're feeling this way. 💔 You're not alone — please consider talking to someone you trust.",
        "Your feelings matter. 🫂 If you're struggling, reach out to a friend, family member, or a mental health professional. Help is available. ❤️",
        "You are important. Please don’t keep this to yourself — talk to someone. 🙏",
        "You are not alone in this. 💙 It's okay to seek help and support.",
        "If you're in crisis, reach out to a mental health helpline or a caring friend. You deserve support. 🧠💬",
        "You're going through something heavy — please speak to someone you trust. You're valued. 🌹",
        "It's brave to speak up — keep going, and don’t hesitate to ask for help. 🚨",
        "Even if it doesn’t feel like it now, things can and do get better. You matter. 🌅",
        "I'm here to remind you: You're loved, you're needed, and you're not alone. 💞",
        "Please stay. The world is better with you in it. 🌍💛"
    ]
}


In [7]:
# Function to predict sentiment and respond
def predict_sentiment(user_input):
    # First, check for suicidal intent using keyword-based detection
    emotion_intent = detect_emotion_intent(user_input)

    if emotion_intent == "suicidal":
        sentiment = "suicidal"
    else:
        # Use the sentiment model pipeline
        result = sentiment_pipeline(user_input)
        label = result[0]["label"]

        # Map model labels to positive, neutral, negative
        if "5 stars" in label or "4 stars" in label:
            sentiment = "positive"
        elif "3 stars" in label:
            sentiment = "neutral"
        else:
            sentiment = "negative"

    # Pick a response from the predefined list
    response = intent_responses[sentiment][0]

    return f"Sentiment: {sentiment.capitalize()}\nResponse: {response}"


In [8]:
interface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=3, placeholder="How are you feeling today?"),
    outputs="text",  # <- Fix here
    title="💬 Sentiment Analysis using BERT and Transformers",
    description="An AI-powered sentiment analysis tool using BERT and Transformers to detect emotional states and provide supportive responses. ❤️"
)


In [9]:
interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ba534bbe8f6c5ca7a3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
